In [1]:
!pip install openai
!pip install python-dotenv
!pip install spacy-langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 998.1/998.1 kB 52.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 17.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993415 sha256=9cae4bc9b2588b19e2bc2dcaa80930b8c677ce892fce54b5bee1b9b218b6411b
  Stored in directory: /root/.cache/pip/wheels/68/75/38/74d00301173db8c126a3242f0315a6e0d4824018e9b6e5d6e2
Successfully built langdetect
  Attempting uninstall: attrs
    Found existing installation: attrs 18.2.0
    Uninstalling attrs-18.2.0:
      Successfully uninstalled attrs-18.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.


In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import pandas as pd
import openai
import os

In [4]:
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
df = pd.read_csv('fair-data-all.csv')
len(df)

556

In [6]:
df = df.drop_duplicates(subset=['Title'])
len(df)

555

In [7]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

@Language.factory("language_detector")
def get_lang_detector(nlp, name):
   return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('language_detector', last=True)

In [8]:
for index, row in df.iterrows():
    df.at[index, 'Lang'] = nlp(row['Title'])._.language['language']

In [9]:
df = df[df.Lang == 'en']
len(df)

549

In [123]:
df.iloc[[62,355,406,464]].DOI

62            10.1162/dint_x_00187
358    10.1016/j.jphys.2017.02.002
410           10.2147/clep.s105933
469     10.1186/s12939-023-01855-0
Name: DOI, dtype: object

In [10]:
df2 = df[df['Abstract'].str.contains('FAIR')].drop(62).drop(355).drop(406).drop(464)
len(df2)

31

In [95]:
intro = "You need to decide in which category the following text falls into \
according to its content. Possible choices are: \
'Overview and Adoption of FAIR Principles' (that could include general reviews and surveys, case studies, guidelines, challenges and barriers to adoption),\
'FAIRification of data' (that could include methodologies for data storage, access, curation, preservation, archiving, security, privacy),\
'Infrastructure and Tools' (that could include data repositories and platforms, metadata standards and ontologies).\n\nText: "

In [96]:
for index, row in df2.iterrows():
    text = intro + row['Abstract'] + "\nCategory:"
    response = openai.Completion.create(model="text-davinci-003", 
                                    prompt=text, 
                                    temperature=0, 
                                    max_tokens=20)

    df2.at[index, 'Category'] = response["choices"][0]["text"].strip()

In [97]:
df2['Category'].value_counts()

Overview and Adoption of FAIR Principles    18
Infrastructure and Tools                     8
FAIRification of data                        5
Name: Category, dtype: int64

In [99]:
df2[['Category', 'DOI', 'Title']].sort_values(['Category','Title'])

,Category,DOI,Title
2,FAIRification of data,10.1016/j.dib.2020.106424,An Exploratory Assessment of a Multidimensiona...
326,FAIRification of data,10.1186/s13326-022-00263-7,Applying the FAIR principles to data in a hosp...
515,FAIRification of data,10.23889/ijpds.v7i3.1887,Augmenting laboratory COVID serology data gran...
187,FAIRification of data,10.1162/dint_a_00182,FAIR Machine Learning Model Pipeline Implement...
124,FAIRification of data,10.1371/journal.pone.0212291,Term sets: A transparent and reproducible repr...
87,Infrastructure and Tools,10.2196/42822,A Data Transformation Methodology to Create Fi...
524,Infrastructure and Tools,10.1093/eurpub/ckac129.468,An enhanced version of the PHIRI infrastructur...
456,Infrastructure and Tools,10.1038/s41431-020-0637-5,"Connecting data, tools and people across Europ..."
166,Infrastructure and Tools,10.1016/j.radonc.2019.11.019,Distributed learning on 20 000+ lung cancer pa...
505,Infrastructure and Tools,10.1038/s41597-022-01265-x,FAIR Genomes metadata schema promoting Next Ge...


In [124]:
df2.to_csv('fair-data-only.csv')

In [101]:
df2['Abstract'].str.contains('privacy|security', regex=True).sum()

5

In [102]:
df3 = df2[df2['Abstract'].str.contains('privacy|security', regex=True)]

In [103]:
df3.DOI

36              10.3390/app12094336
87                    10.2196/42822
147       10.1016/j.jbi.2020.103421
166    10.1016/j.radonc.2019.11.019
237            10.15265/iy-2017-024
Name: DOI, dtype: object